In [13]:
# !wget "https://www.dropbox.com/s/4jw31k5mlzcmgis/genres.tar.gz?"
# !tar xvf "/content/genres.tar.gz?" -C "/content/"

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
import matplotlib.pyplot as plt
from tensorflow import keras
from librosa import feature
import tensorflow as tf
from glob import glob
import pandas as pd
import numpy as np
import librosa
import csv

import shutup
shutup.please()

Feature Extraction from Audio data and saving them in their respective .csv 📁 file 🔉

In [ ]:
genre = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

header =[
  'chroma_stft',
  'spectral_centroid',
  'spectral_bandwidth',
  'spectral_rolloff',
  
  'chroma_stft_var',
  'spectral_centroid_var',
  'spectral_bandwidth_var',
  'spectral_rolloff_var',
  
  'rmse',
  'zero_crossing_rate',
  'librosa.feature.spectral_contrast',
  'librosa.feature.spectral_flatness',
  'librosa.feature.mfcc',

  'rmse_var',
  'zero_crossing_rate_var',
  'librosa.feature.spectral_contrast_var',
  'librosa.feature.spectral_flatness_var',
  'librosa.feature.mfcc_var'
  ]

fn_list_i = [
            feature.chroma_stft, feature.spectral_centroid,
            feature.spectral_bandwidth, feature.spectral_rolloff
            ]

fn_list_ii = [librosa.feature.rms,
              feature.zero_crossing_rate,
              librosa.feature.spectral_contrast,
              librosa.feature.spectral_flatness,
              librosa.feature.mfcc]
  

def get_feature_vector(y,sr): 
    feat_vect_i = [ np.mean(funct(y,sr)) for funct in fn_list_i]
    feat_vect_i_var = [ np.var(funct(y,sr)) for funct in fn_list_i]

    feat_vect_ii = [np.mean(funct(y)) for funct in fn_list_ii]
    feat_vect_ii_var = [np.var(funct(y)) for funct in fn_list_ii]

    feature_vector = feat_vect_i + feat_vect_i_var + feat_vect_ii + feat_vect_ii_var 
    return feature_vector

for nowvar in genre:

  norm_data_dir = '/content/genres/'+nowvar+'/'
  norm_audio_files = glob(norm_data_dir + '*.au')

  norm_audios_feat = []
  for file in norm_audio_files:
    y , sr = librosa.load(file, sr=None)
    feature_vector = get_feature_vector(y, sr)
    norm_audios_feat.append(feature_vector)

  norm_output = nowvar+'.csv'

  with open(norm_output,'+w') as f:
    csv_writer = csv.writer(f, delimiter = ',')
    csv_writer.writerow(header)
    csv_writer.writerows(norm_audios_feat)

Reading .csv file and adding a new column type ✔

In [ ]:
blues = pd.read_csv("/content/blues.csv")
classical = pd.read_csv("/content/classical.csv")
country = pd.read_csv("/content/country.csv")
disco = pd.read_csv("/content/disco.csv")
hiphop = pd.read_csv("/content/hiphop.csv")
jazz = pd.read_csv("/content/jazz.csv")
metal = pd.read_csv("/content/metal.csv")
pop = pd.read_csv("/content/pop.csv")
reggae = pd.read_csv("/content/reggae.csv")
rock = pd.read_csv("/content/rock.csv")

blues['label'] = 'blues'
classical['label'] = 'classical'
country['label'] = 'country'
disco['label'] = 'disco'
hiphop['label'] = 'hiphop'
jazz['label'] = 'jazz'
metal['label'] = 'metal'
pop['label'] = 'pop'
reggae['label'] = 'reggae'
rock['label'] = 'rock'

Creating The Main Dataframe for training our Model 🚩

In [ ]:
final_df = pd.DataFrame(columns = rock.columns)

In [ ]:
genres = [blues, 
          classical, 
          country, 
          disco, 
          hiphop, 
          jazz,
          metal, 
          pop,
          reggae,
          rock]

final_df = pd.concat(genres)

In [ ]:
final_df.shape

In [ ]:
final_df.sample(20)

Encoding labels of Target column ✈

In [ ]:
convertor = LabelEncoder()
class_list = final_df.iloc[:,-1]
y = convertor.fit_transform(class_list)

Fitting Rest of the columns in StanderdScaler (Using fit_transform for that)

In [ ]:
fit = StandardScaler()
X = fit.fit_transform(np.array(final_df.iloc[:,:-1], dtype = float))

Splitting Our Dataset for Training (x_train & y_train) and Testing (y_train & y_test)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

Creating TrainModel Function for training our RNN model.

In [ ]:
def trainModel (model, epochs, optimizer, verbose):
  batch_size = 128

  model.compile(optimizer,
                loss='sparse_categorical_crossentropy',
                metrics='accuracy')

  return model.fit(x_train, y_train,
                 validation_data=(x_test, y_test),
                 epochs=epochs, batch_size=batch_size, verbose=verbose)

Our Sequential Model ⚖

In [ ]:
model = keras.models.Sequential([
                             keras.layers.Dense(512, activation='relu', input_shape=(x_train.shape[1],)),
                             keras.layers.Dropout(0.2),

                             keras.layers.Dense(256, activation='relu'),
                             keras.layers.Dropout(0.2),

                             keras.layers.Dense(128, activation='relu'),
                             keras.layers.Dropout(0.2),

                             keras.layers.Dense(64, activation='relu'),
                             keras.layers.Dropout(0.2),

                             keras.layers.Flatten(),
                             keras.layers.Dense(10, activation='softmax')
                          ])
print(model.summary())
model_history = trainModel(model=model, epochs=600, optimizer='adam', verbose=0)

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test, batch_size=128)
print (f"\nThe test loss is : {round(test_loss, 2)}")
print(f"The best test accuracy is : {round(test_acc*100, 2)}%")

In [ ]:
model.summary()

In [ ]:
def makeAudio_to_Data (path):

  norm_audios_feat = []

  y , sr = librosa.load(file, sr=None)
  feature_vector = get_feature_vector(y, sr)
  norm_audios_feat.append(feature_vector)

  newDataframe = pd.DataFrame(norm_audios_feat, columns=header)

  fit2 = StandardScaler()
  numpyArray = fit2.fit_transform(np.array(final_df.iloc[:,:], dtype = float))

  return numpyArray, newDataframe

data = makeAudio_to_Data ('/content/POP SMOKE - DIOR (OFFICIAL VIDEO).mp3')
